In [1]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import pytz

In [8]:
def get_flights_data(icao_codes):
    df_airports = get_airports_data(icao_codes).iloc[:,0:5]                                    #call function to get the airport details
    df_flights = flight_dataframe(icao_codes)                                                  #call function to get the flights details
    data = df_flights.merge(right=df_airports, on='icao', how='inner')                         #merge to dataframes to get the final DF
    data.rename(columns={'icao':'arrival_icao','iata':'arrival_iata',
    'fullName':'arrival_airport','municipalityName':'arrival_city'}, inplace=True)             #rename the columns
    data['departure_time_local'] = pd.to_datetime(data['departure_time_local'].str[:16])       #convert the departure time to datetime
    data['arrival_time_local'] = pd.to_datetime(data['arrival_time_local'].str[:16])           #convert the departure time to datetime
    data=data.reindex(columns=['departure_airport','departure_time_local','arrival_city','arrival_icao','arrival_iata','arrival_airport',
    'arrival_time_local','airline','flight_number','flight_status'])
    return data

def get_airports_data(icao_codes):
    list_for_df = []

    for i in icao_codes:
        url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{i}"
        headers = {
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
            "X-RapidAPI-Key": "API Key"                           #API Key
        }
        response = requests.request("GET", url, headers=headers)
        list_for_df.append(pd.json_normalize(response.json()))

    df_airports = pd.concat(list_for_df, ignore_index=True)
    return df_airports

def flight_dataframe(icao_codes):
    flight_dic = {'departure_airport':[], 'departure_time_local':[], 'icao':[], 'arrival_time_local':[],               #Initilize the dictionary
                 'flight_status':[], 'flight_number':[], 'airline':[]}
    berlinTz = pytz.timezone("Europe/Berlin") 
    today = datetime.now(berlinTz).date()
    tomorrow= today + timedelta(days=1)

    for icao_code in icao_codes:
        times = [["00:00","11:59"],["12:00","23:59"]]

        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao_code}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"false","withCargo":"false","withPrivate":"true","withLocation":"false"}
            headers = {
                "X-RapidAPI-Key": "API_Key",                         #API Key
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            response2 = requests.request("GET", url, headers=headers, params=querystring)
            response2.raise_for_status()                                                                        #response status
            result2 = response2.json()                                                                          # json format

            for item in result2['arrivals']:
                flight_dic['departure_airport'].append(item['departure']['airport'].get('name',None))                    # departure airport
                flight_dic['departure_time_local'].append(item['departure'].get('scheduledTimeLocal', None))        # departure time
                flight_dic['icao'].append(icao_code)                                                            # arrival airport
                flight_dic['arrival_time_local'].append(item['arrival'].get('scheduledTimeLocal',None))                        # arrival time 
                flight_dic['flight_status'].append(item['status'])                                              # flight status
                flight_dic['flight_number'].append(item['number'])                                                  # flight number
                flight_dic['airline'].append(item['airline'].get('name',None))                                           # airline
    df_flights=pd.DataFrame(flight_dic)
    return df_flights

In [ ]:
#list_airports=['BER', 'LHR', 'CDG', 'MAD', 'MXP', 'MUC']        # [Berlin, London, Paris, Madrid, Milan, Munich]
list_airports=['EDDB', 'EGLL', 'LFPG', 'LEMD', 'LIMC', 'EDDM']
df=get_flights_data(list_airports)
df

In [10]:
schema="db_weather_flights"   # name of the database you want to use here
host="127.0.0.1"              # to connect to your local server
user="root"
password=" Password " # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [11]:
df.to_sql('flights',              # 'cities'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=con,            # con-> connection string;
              index=False) 

2373